In [1]:
import numpy as np
import os
import pandas as pd
import SimpleFunctions as SF
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
# Optimization Parameters
ConstitutiveModels = ['Ogden', 'Neo-Hookean']
ConstitutiveModel = ConstitutiveModels[0]

BCsTypes = ['Ideal', 'Fixed']
BCsType  = BCsTypes[1]


NumberElementsTested = [1, 2, 3, 5, 10, 20]
NumberElements = NumberElementsTested[5]

# Folder for the results
Optim_Path = os.path.join('OptimizationResults', BCsType, ConstitutiveModel)

OptiName = str(NumberElements) + 'Elements.txt'
MarcName = str(NumberElements) + 'Elements-M.txt'

# Cost Function Data
CostData = pd.read_csv('CostFunctionGrid/Results.csv', sep=',', decimal='.') 
OptiData = pd.read_csv(os.path.join(Optim_Path, OptiName), sep=' ', decimal='.')
MarcData = pd.read_csv(os.path.join(Optim_Path, MarcName), sep=' ', decimal='.')

# Row with the minimum cost
print('Min cost point for:')
print(CostData.loc[CostData['TotalCostNormMax'].idxmin()])

Nus    = CostData.Nu
Mus    = CostData.Mu
Alphas = CostData.Alpha
Costs  = CostData.TotalCostNormMax

DM, MuM, AlphaM = 1.026, 0.458, -59.367
NuM = (3-DM*MuM)/(DM*MuM+6)
Marcdf = pd.DataFrame([NuM, MuM, AlphaM])

Threshold = 1E-0
Subsampling = 1

Vmin = Costs.min()
Vmax = Threshold * (Costs.max() - Costs.min())

# 3D Plot
%matplotlib widget
plt.rc('figure', figsize=[12,7])
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

#img = ax.scatter(Nus, Mus, Alphas, c=Costs, cmap=plt.jet(), vmin=0, vmax=0.1)
CostImage = ax.scatter(Nus[0::Subsampling], Mus[0::Subsampling], Alphas[0::Subsampling], c=Costs[0::Subsampling], cmap=plt.gray(), marker = '.', vmin=Vmin, vmax=Vmax, label='Points tested')
ColorBar=plt.colorbar(CostImage)
ColorBar.set_label('Cost (-)')
ax.set_xlabel('Nu (-)')
ax.set_ylabel('Mu (kPa)')
ax.set_zlabel('Alpha (-)')
ax.plot([Nus[CostData['TotalCostNormMax'].idxmin()]], [Mus[CostData['TotalCostNormMax'].idxmin()]], [Alphas[CostData['TotalCostNormMax'].idxmin()]], marker = 'o', color='k', label='Grid min cost point')
ax.plot([OptiData.Nu[0]], [OptiData.Mu[0]], [OptiData.Alpha[0]], marker = 'o', color='m', label='Budday starting point')
ax.plot(OptiData.Nu, OptiData.Mu, OptiData.Alpha, marker = None, color='m', label='Optimization path')
ax.plot([OptiData.Nu[len(OptiData)-1]], [OptiData.Mu[len(OptiData)-1]], [OptiData.Alpha[len(OptiData)-1]], marker = 'o', color='r', label='Optimization result')
ax.plot([MarcData.Nu[0]], [MarcData.Mu[0]], [MarcData.Alpha[0]], marker = 'o', color='c', label='Marc starting point')
ax.plot(MarcData.Nu, MarcData.Mu, MarcData.Alpha, marker = None, color='c', label='Optimization path')
ax.plot([MarcData.Nu[len(MarcData)-1]], [MarcData.Mu[len(MarcData)-1]], [MarcData.Alpha[len(MarcData)-1]], marker = 'o', color='b', label='Optimization result')
ax.plot(Marcdf.loc[0], Marcdf.loc[1], Marcdf.loc[2], marker = 'o', color='g', label='Marc optimization result')
ax.legend()
plt.show()

Min cost point for:
Alpha                             -24
BCsType                         Fixed
CompressionCostNoNorm       0.0296841
CompressionCostNormMax      0.0531391
CompressionCostStep           9.18372
ConstitutiveModel               Ogden
Mu                                0.6
Nu                               0.41
NumberElements                     20
SimpleShearCostNoNorm      0.00813879
SimpleShearCostNormMax       0.116729
SimpleShearCostStep            2.2651
TensileCostNoNorm         0.000807557
TensileCostNormMax           0.032357
TensileCostStep               1196.05
TotalCostNoNorm             0.0128768
TotalCostNormMax            0.0674084
TotalCostStep                   402.5
Name: 27848, dtype: object


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
# Filtering

MaxRow = CostData['TotalCostNormMax'].idxmax()

Threshold = 1
MaxCost = CostData.TotalCostNormMax.max()

FilteredCost = CostData.dropna()

# for Point in range(len(FilteredCost)):
#     CostValue = FilteredCost.TotalCostNormMax[Point]
#     if CostValue > MaxCost * Threshold:
#         FilteredCost = FilteredCost.drop([Point])
        
# Filtered 3D Plot
%matplotlib widget
plt.rc('figure', figsize=[12,7])
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

Nus    = FilteredCost.Nu
Mus    = FilteredCost.Mu
Alphas = FilteredCost.Alpha
Costs  = FilteredCost.TotalCostNormMax

# img = ax.scatter(Nus, Mus, Alphas, c=Costs, cmap=plt.jet(), vmin=0, vmax=0.1)
CostImage = ax.scatter(Nus, Mus, Alphas, c=Costs, cmap=plt.viridis(), marker = '.', label='Points tested')
ColorBar=plt.colorbar(CostImage)
ColorBar.set_label('Cost (-)')
ax.set_xlabel('Nu (-)')
ax.set_ylabel('Mu (kPa)')
ax.set_zlabel('Alpha (-)')
ax.plot([Nus[FilteredCost['TotalCostNormMax'].idxmin()]], [Mus[FilteredCost['TotalCostNormMax'].idxmin()]], [Alphas[FilteredCost['TotalCostNormMax'].idxmin()]], marker = 'o', color='k', label='Grid min cost point')
ax.plot([OptiData.Nu[len(OptiData)-1]], [OptiData.Mu[len(OptiData)-1]], [OptiData.Alpha[len(OptiData)-1]], marker = 'o', color='r', label='Optimization result')
ax.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
CostData.dropna()

,Alpha,BCsType,CompressionCostNoNorm,CompressionCostNormMax,CompressionCostStep,ConstitutiveModel,Mu,Nu,NumberElements,SimpleShearCostNoNorm,SimpleShearCostNormMax,SimpleShearCostStep,TensileCostNoNorm,TensileCostNormMax,TensileCostStep,TotalCostNoNorm,TotalCostNormMax,TotalCostStep
4,-14.0,Fixed,0.573213,1.026139,4.769848,Ogden,0.16,0.295,20.0,0.080508,1.154676,5.235875,0.038062,1.525069,496.437200,0.230594,1.235294,168.814307
13,-20.0,Fixed,0.470750,0.842714,3.204773,Ogden,0.18,0.295,20.0,0.071667,1.027877,4.742739,0.035301,1.414439,308.084656,0.192573,1.095010,105.344056
52,-17.0,Fixed,0.657123,1.176351,3.535038,Ogden,0.10,0.295,20.0,0.087982,1.261867,5.340791,0.041526,1.663850,144.001162,0.262210,1.367356,50.958997
58,-11.0,Fixed,0.687605,1.230919,4.334974,Ogden,0.10,0.295,20.0,0.090706,1.300930,5.619072,0.043181,1.730146,330.440243,0.273830,1.420665,113.464763
66,-17.0,Fixed,0.618764,1.107684,3.450435,Ogden,0.12,0.295,20.0,0.084639,1.213915,5.230610,0.040049,1.604681,199.679730,0.247817,1.308760,69.453592
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23953,-42.0,Fixed,6.065566,10.858299,15.322597,Ogden,0.58,0.440,20.0,0.124627,1.787436,3.698482,0.004614,0.184892,270.947492,2.064936,4.276876,96.656190
23954,-48.0,Fixed,16.057634,28.745643,35.545390,Ogden,0.56,0.420,20.0,0.451404,6.474188,9.453272,0.000342,0.013694,236.443823,5.503127,11.744509,93.814162
23955,-54.0,Fixed,37.936310,67.911852,78.696660,Ogden,0.50,0.425,20.0,1.155912,16.578470,21.159273,0.000173,0.006938,144.785991,13.030798,28.165753,81.547308
23956,-40.0,Fixed,3.894837,6.972360,11.237991,Ogden,0.60,0.405,20.0,0.077717,1.114639,3.067619,0.000387,0.015507,439.151404,1.324314,2.700835,151.152338


In [9]:
CostData

,Alpha,BCsType,CompressionCostNoNorm,CompressionCostNormMax,CompressionCostStep,ConstitutiveModel,Mu,Nu,NumberElements,SimpleShearCostNoNorm,SimpleShearCostNormMax,SimpleShearCostStep,TensileCostNoNorm,TensileCostNormMax,TensileCostStep,TotalCostNoNorm,TotalCostNormMax,TotalCostStep
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-14.0,Fixed,0.573213,1.026139,4.769848,Ogden,0.16,0.295,20.0,0.080508,1.154676,5.235875,0.038062,1.525069,496.4372,0.230594,1.235294,168.814307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25284,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25287,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
